# Notes about SNAP
1. snap.GetBetweennessCentr(PNGraph, Nodes, Edges, InDist=False) equals to snap.GetBetweennessCentr(PUNGraph, Nodes, Edges)
2. snap.GetDegreeCentr(UG, nid) == float(n.GetDeg())/(size-1)
3. http://snap.stanford.edu/class/cs224w-2013/recitation/snappy/Snappy-tutorial-2013.pdf

# Notes about func intxn_from_segs(segs, directionality_column=None):
    
    # check whether the intersection output is correct, by picking some examples for both B, FT, TF. 
    1. FT/0 type: DC STREETSEGID = 876(-77.01239193659748 38.950836102255), checked; 14713(-76.9902413493931 38.8773984704994), checked; 8891(-77.01366204332678 38.899295772364), checked
    2. TF/1 type: DC STREETSEGID = 13161(-77.01163764014346 38.953383268855), checked; 8292(-77.0100561260908 38.9555132405171), checked; 92(-77.01618238255742 38.896139800171), checked
    3. B/2 type: DC STREETSEGID = 10395(-77.0116155077942 38.9556194835925), checked; 10090(-77.0116172263626 38.9576690437764), checked; 15306(-77.01885842791256 38.836994168269), checked; 15284(-76.99377942384207 38.875606235068)



1. feature for segs as node:
    - N
        - [x] in/out degree
        - [x] betweenness centrality(https://snap.stanford.edu/snappy/doc/reference/GetBetweennessCentr.html)
        - [x] closeness centrality(https://snap.stanford.edu/snappy/doc/reference/GetClosenessCentr.html)
        - [x] Farness centrality(https://snap.stanford.edu/snappy/doc/reference/GetFarnessCentr.html)
        - [x] page rank(https://snap.stanford.edu/snappy/doc/reference/GetPageRank.html)
        - [x] Hits score(https://snap.stanford.edu/snappy/doc/reference/GetHits.html)
        - [x] node eccentricity(https://snap.stanford.edu/snappy/doc/reference/GetNodeEcc.html)
    - UN
        - [x] degree centrality(https://snap.stanford.edu/snappy/doc/reference/GetDegreeCentr.html)
        - [x] eigenvector centrality(https://snap.stanford.edu/snappy/doc/reference/GetEigenVectorCentr.html)
        - [x] betweenness centrality(https://snap.stanford.edu/snappy/doc/reference/GetBetweennessCentr.html)
        - [x] closeness centrality(https://snap.stanford.edu/snappy/doc/reference/GetClosenessCentr.html)
        - [x] Farness centrality(https://snap.stanford.edu/snappy/doc/reference/GetFarnessCentr.html)
        - [x] page rank(https://snap.stanford.edu/snappy/doc/reference/GetPageRank.html)
        - [x] Hits score(https://snap.stanford.edu/snappy/doc/reference/GetHits.html)
        - [x] node eccentricity(https://snap.stanford.edu/snappy/doc/reference/GetNodeEcc.html)
        - [x] articulation pointshttps://snap.stanford.edu/snappy/doc/reference/GetArtPoints.html
        - [x] bridge(https://snap.stanford.edu/snappy/doc/reference/GetEdgeBridges.html)

In [1]:
import snap
import geopandas as gp
import pandas as pd

In [63]:
import src.constants;reload(src.constants)
import src.geom_helper; reload(src.geom_helper)
from src.geom_helper import intxn_from_segs
from src.constants import fn_intxn_dc, fn_feature_seg_as_node, fn_segments_dc, index_seg, fn_feature_seg_as_node_dc
from src.constants import fn_segments_ph, epsg_ph, latlon_crs, fn_feature_seg_as_node_ph

In [9]:
import src.ftr_network; reload(src.ftr_network)
from src.ftr_network import *

In [4]:
data_dir = 'data/'

In [6]:
path_segs_ph = data_dir + fn_segments_ph
intxn_directed = intxn_from_segs(path_segs_ph, directionality_column='ONEWAY')
intxn_directed.shape

intersction project {(0.0, 1.0): 117, (1.0, 0.0): 65, (0.0,): 100579, (): 1422, (1.0,): 99293}


(128845, 2)

In [64]:
path_segs_ph = data_dir + fn_segments_ph
path_segs_dc = data_dir + fn_segments_dc
path_ftr_SgAsNd_dc = data_dir + fn_feature_seg_as_node_dc
path_ftr_SgAsNd_ph = data_dir + fn_feature_seg_as_node_ph

In [6]:
def ftr_network_SgAsNd(path_segs, path_ftr_seg_as_node, directionality_column=None, is_dird=False):
    from src.constants import index_seg
    print 'ftr_network sgasnd'
    intxn_matrix = intxn_from_segs(path_segs, directionality_column=directionality_column)
    print 'got intxn_matrix'
    edges = intxn_matrix.values
    nodes = pd.unique(edges.flatten())
    features_SgAsNd = ftr_segs_as_nodes(nodes, edges, is_dird)
    print 'got ftr sgasnd'
    df_ftr = pd.DataFrame.from_dict(features_SgAsNd).T
    df_ftr.index.name = index_seg
    df_ftr.to_csv(path_ftr_seg_as_node)
    

In [ ]:
ftr_network_SgAsNd(path_segs_dc, path_ftr_SgAsNd_dc, 'DIRECTIONALITY', True)

In [ ]:
ftr_network_SgAsNd(path_segs_ph, path_ftr_SgAsNd_ph, 'ONEWAY', True)


def ftr_segs_as_nodes(nodes, edges, is_dird=False):

    # TODO: remove arg:is_dird with directionality_column
    # TODO: modify the feature name and function name, add SgAsNd
    # TODO: reconsider the function name, because processs of building network of SgAsNd and SgAsEg is quite different


In [47]:
from shapely.ops import linemerge
path_segs_dc = data_dir + fn_segments_dc
segs_dc = gp.read_file(path_segs_dc)

segs_dc.geometry = segs_dc.geometry.apply(lambda x: x if x.type!='MultiLineString' else linemerge(x))

intxn_directed_dc = intxn_from_segs(segs_dc, directionality_column='DIRECTIONALITY', debug=True)
intxn_directed_dc.shape

intersction project {(0.0, 1.0): 43, (1.0, 0.0): 44, (0.0,): 33141, (): 488, (1.0,): 32690}


(65918, 14)

In [ ]:
def ftr_netw_SgAsEg(path_segs, path_ftr_seg_as_edge, directionality_column)

In [34]:
intxn_dcff = pd.read_csv(path_intxn_dc)

In [38]:
intxn_dcff[intxn_dcff.STREET2SEGID==8573]

,﻿X,Y,OBJECTID,MARID,INTERSECTIONID,STREET1ID,STREET2ID,ST1NAME,ST1TYPE,ST1QUAD,...,REFX,REFY,NATIONALGRID,STREET1SEGID,STREET2SEGID,NODEID,INTERSECTION_TYPE,SOURCE,LATITUDE,LONGITUDE


In [32]:
path_intxn_dc = data_dir+fn_intxn_dc
path_segs_dc = data_dir + fn_segments_dc
path_ftr_seg_as_node_dc = data_dir + fn_feature_seg_as_node

In [5]:
df_dc_seg_as_node = ftr_for_dc(path_intxn_dc, path_segs_dc, path_ftr_seg_as_node_dc)

node size = 13443, directed edges = 18006, undirected edges = 14147
